In [1]:
from typing import Any

import lightning.pytorch as pl
import torch
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow
from torch.utils.data import DataLoader, Dataset

from src.ml.data.splitting import create_data_splits
from src.ml.modeling import (
    model_factory,
    optimizer_factory,
)
from src.ml.utils.set_seed import set_seed

from datetime import datetime
from loguru import logger
from pathlib import Path
from shutil import copy, copytree, rmtree

import yaml

from src.ml.train_neural_network import train_neural_network
from src.ml.data import data_sets_factory
from src.ml.preprocessing import preprocessing_factory
from src.ml.utils.set_seed import set_seed

In [2]:
CONFIG_FILE = Path("src/ml/config.yaml")

In [3]:
torch.set_default_device(torch.device("cpu"))

In [4]:
# load config file

logger.info("Loading config file.")

with open(CONFIG_FILE, "r") as f:
    config = yaml.safe_load(f)

2024-12-13 02:46:08.052 | INFO     | __main__:<module>:3 - Loading config file.


In [5]:
# load data

logger.info("Loading data.")

data_sets = data_sets_factory(**config["data_set"])
logger.info("Loaded {} data sets.", len(data_sets))

# preprocess data

logger.info("Start preprocessing.")

for preprocessing_step in config["preprocessing"]:
    logger.info("Perform {} preprocessing.", preprocessing_step["name"])

    transform = preprocessing_factory(**preprocessing_step)
    data_sets = [transform(data_set) for data_set in data_sets]

dataset = data_sets[0]

2024-12-13 02:46:08.058 | INFO     | __main__:<module>:3 - Loading data.
2024-12-13 02:46:08.058 | INFO     | __main__:<module>:6 - Loaded 1 data sets.
2024-12-13 02:46:08.059 | INFO     | __main__:<module>:10 - Start preprocessing.


In [6]:
train_dataset, val_dataset, test_dataset = create_data_splits(
    dataset, **config["training"]["splitting_config"]
)

In [7]:
train_loader = DataLoader(train_dataset, **config["training"]["dataloader_config"])
test_loader = DataLoader(test_dataset,batch_size=1)
val_loader = DataLoader(val_dataset)

optimizer = optimizer_factory(**config["training"]["optimizer_config"])
model = model_factory(
    optimizer=optimizer,
    dim=len(train_dataset[0]["branch_lengths"]),
    **config["training"]["model_config"],
)

In [8]:
from src.ml.modeling.conditional_tree_flow import ConditionalTreeFlow

model = ConditionalTreeFlow.load_from_checkpoint("ml_data/models/debug_simple_conditional_flow/epoch=99-val_loss=-10310.73.ckpt")

In [9]:
sample = next(iter(test_loader))
sample

{'branch_lengths': tensor([[-0.0007,  0.0005,  0.0022, -0.0011,  0.0010,  0.0013, -0.0008,  0.0003,
          -0.0014, -0.0001]]),
 'clades_one_hot': tensor([[1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
          0., 1.]])}

In [10]:
latent = model.forward(sample)
latent

{'z': tensor([[-1.5791,  0.5155,  1.8013, -0.6198,  0.4415, -1.0975, -1.1850, -0.5833,
          -1.3167,  0.5726]], grad_fn=<MulBackward0>),
 'context': tensor([[1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
          0., 1.]]),
 'log_dj': tensor([55.8359], grad_fn=<AddBackward0>)}

In [11]:
output = model.inverse(latent)
output

{'branch_lengths': tensor([[-0.0007,  0.0005,  0.0022, -0.0011,  0.0010,  0.0013, -0.0008,  0.0003,
          -0.0014, -0.0001]], grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
          0., 1.]]),
 'log_dj': tensor([55.8359], grad_fn=<AddBackward0>)}

In [51]:
prior = model.prior.sample((10,))
latent["z"] = prior.clone()
output = model.inverse(latent)
output

{'branch_lengths': tensor([ 1.0651,  0.0799, -0.5086,  0.0533,  0.1438,  1.4824,  0.0354,  0.1901,
          0.2053, -0.2005], grad_fn=<SubBackward0>),
 'clades_one_hot': tensor([[1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
          0., 1.]]),
 'log_dj': tensor([55.8359], grad_fn=<AddBackward0>)}